# Movie_DataQuest - Search For The Best Single Factor

In [ ]:
# Dependencies
import requests
from config import api_key

### Steve's Code Goes Here

In [ ]:
#Get MovieLens IMDB data

In [ ]:
# Get TMDB data especially with respect to box office 

In [ ]:
# Merge MovieLens with TMDB

### Jim's Code Goes Here

In [ ]:
# Get 900 random movie sample from the OMDB

In [ ]:
# Merge based on movie code to produce testible db 

### Kelly's Code Here

In [ ]:
# Correlate Box Office v. Run time

In [ ]:
# Correlate Box Office v. Budget

Awards / nominations
MPAA Rating


### Philips' Code Goes Here

In [ ]:
# Correlate Box Office v. Critical reviews

In [ ]:
# Correlate Box Office v. User reviews

### Jim's Correlation Section Goes Here

In [ ]:
# Correlate Box Office v Awards / nominations

In [ ]:
# option:  Correlate Box Office v MPAA Rating

### Steve's Correlation Section Goes Here

In [ ]:
# option  Correlate Box Office v MPAA Rating

### BONUS: Perform t-test across movie genres

In [ ]:
# option: Steve performs t-test across movie genre 
# and Jim does both Awards & MPAA Rating correlations